# 序列化读写模型

使用``nd.save``,``nd.load``可以方便的读写``NDArray``，使用``save_params``和``load_params``可以方便的读写模型参数。

In [13]:
import mxnet as mx

from mxnet import nd
from mxnet import gluon
from mxnet import autograd
mx.random.seed(1)

ctx = mx.cpu()

## 1. ``nd.save``,``nd.load``

In [14]:
a = nd.array([1,2,3,4])
b = nd.arange(4).reshape((2,2))
filename = "serialization/data.params"
nd.save(filename, [a, b])

In [15]:
nd.load(filename)

[
 [ 1.  2.  3.  4.]
 <NDArray 4 @cpu(0)>, 
 [[ 0.  1.]
  [ 2.  3.]]
 <NDArray 2x2 @cpu(0)>]

In [16]:
dic1 = {'a': a, 'b': b}
filename = 'serialization/dict_data.params'
nd.save(filename, dic1)

In [17]:
nd.load(filename)

{'a': 
 [ 1.  2.  3.  4.]
 <NDArray 4 @cpu(0)>, 'b': 
 [[ 0.  1.]
  [ 2.  3.]]
 <NDArray 2x2 @cpu(0)>}

## 2.``save_params``, ``load_params``

In [25]:
def get_net():
    net = gluon.nn.Sequential()
    with net.name_scope():
        net.add(gluon.nn.Dense(64))
        net.add(gluon.nn.Dense(10))
    return net

In [26]:
net = get_net()
net.collect_params().initialize(mx.init.Normal(sigma=.1), ctx=ctx)

In [27]:
X = nd.random.normal(shape=(20, 784)) 
net(X)


[[ -4.07271290e+00  -1.68699276e+00   2.05351025e-01   1.73045421e+00
    2.41705418e+00   3.69414735e+00   4.05352211e+00   6.50496304e-01
    1.45572555e+00   4.96757698e+00]
 [  3.82427883e+00  -2.29843116e+00  -5.87191677e+00   1.16461761e-01
    1.43037784e+00  -5.60459077e-01   6.54590428e-01  -1.36524355e+00
    4.61277336e-01   5.97497225e+00]
 [ -2.88993192e+00  -2.31027651e+00   1.14431214e+00  -6.88245833e-01
    1.50681973e+00   1.47523737e+00  -2.22853613e+00   3.76747996e-01
    3.62078339e-01   3.07201743e+00]
 [  3.19320178e+00   5.29005647e-01  -1.50107336e+00   2.09174347e+00
   -4.06555057e-01   3.93732738e+00  -7.38811493e+00  -7.16894090e-01
   -1.12584078e+00   3.68022561e+00]
 [  1.06302917e+00   3.04841518e+00   1.51992750e+00  -1.18692992e-02
   -1.59391797e+00  -3.74656749e+00  -4.21454620e+00   7.02066123e-02
    3.14034824e-03  -1.88868296e+00]
 [  3.84808278e+00   1.75913382e+00  -1.24975137e-01   1.05135918e+00
   -2.44430587e-01  -2.01775837e+00  -4.5038

In [28]:
filename = "serialization/mlp.params"
net.save_params(filename)

接下来我们构建一个新的MLP，但我们通过``load_params``来加载参数

In [29]:
net2 = get_net()
net2.load_params(filename, ctx=ctx)
net2(X)


[[ -4.07271290e+00  -1.68699276e+00   2.05351025e-01   1.73045421e+00
    2.41705418e+00   3.69414735e+00   4.05352211e+00   6.50496304e-01
    1.45572555e+00   4.96757698e+00]
 [  3.82427883e+00  -2.29843116e+00  -5.87191677e+00   1.16461761e-01
    1.43037784e+00  -5.60459077e-01   6.54590428e-01  -1.36524355e+00
    4.61277336e-01   5.97497225e+00]
 [ -2.88993192e+00  -2.31027651e+00   1.14431214e+00  -6.88245833e-01
    1.50681973e+00   1.47523737e+00  -2.22853613e+00   3.76747996e-01
    3.62078339e-01   3.07201743e+00]
 [  3.19320178e+00   5.29005647e-01  -1.50107336e+00   2.09174347e+00
   -4.06555057e-01   3.93732738e+00  -7.38811493e+00  -7.16894090e-01
   -1.12584078e+00   3.68022561e+00]
 [  1.06302917e+00   3.04841518e+00   1.51992750e+00  -1.18692992e-02
   -1.59391797e+00  -3.74656749e+00  -4.21454620e+00   7.02066123e-02
    3.14034824e-03  -1.88868296e+00]
 [  3.84808278e+00   1.75913382e+00  -1.24975137e-01   1.05135918e+00
   -2.44430587e-01  -2.01775837e+00  -4.5038

可以看到，对于相同的数据，输出结果一样，表明我们成功加载了模型

## 总结

保存模型参数是一件十分重要的事情，有时候我们把它称作“**检查点**”，总的来说，它有以下的诸多好处。

* 我们可以保存以及联合训练模型。
* 一些模型在训练过程中可能在某次epoch处的效果会非常好，如果我们每次epoch过后都保存一次模型参数，我们就可以选择最佳模型。
* 可能我们第一次训练模型的时候还有很多潜在的问题，因此我们可以保存我们训练好的模型并再次运行它们以更加深刻理解我们的模型，而无需每次都从头开始训练。
* 有时候，人们可能想要运行我们的模型，他们不知道如何自己执行训练，或者不能访问合适的训练数据集。checkpoint给了我们一种与他人分享我们的工作的方式。